In [31]:
#the data in the file are result of the analysis mentioned in the Notion doc. [tot] refers to tot users who read that article in the past 6 months
import pandas as pd
df=pd.read_csv("diff%.csv")
df

,page_path,tot,money_diff%,early_diff%,services_diff%,scaleups_diff%,others_diff%
0,/articles/vc-salary-germany-europe/,2000,0.466696,-0.227725,-0.398825,-0.438742,-0.570074
1,/articles/first-time-vc-funds-2022/,1815,0.264700,0.059308,0.023789,-0.404023,-0.479545
2,/articles/startup-tech-company-layoffs/,1525,-0.250762,0.408453,0.383329,1.623109,-0.174098
3,/articles/top-seed-stage-uk-vcs/,1498,0.265302,-0.001146,0.211252,-0.536769,-0.493371
4,/articles/top-series-a-investors-uk/,1254,0.318474,-0.127547,-0.119637,-0.348983,-0.449519
...,...,...,...,...,...,...,...
1167,/articles/servant-economy/,21,-0.023179,1.681511,-1.000000,-1.000000,-0.231074
1168,/articles/sifted-podcast-n26-neobank-controver...,21,-0.316225,1.298438,-1.000000,-1.000000,0.537851
1169,/articles/sifted-summit-cheat-sheet-founders/,21,-0.316225,0.532292,-1.000000,-1.000000,0.922314
1170,/articles/vc-parental-leave/,21,1.051324,-1.000000,-1.000000,-1.000000,-1.000000


In [28]:
#dropping the stuff we don't need and having a look at what we have 
df1 = df.drop(['page_path','tot'],axis=1)

df1

,money_diff%,early_diff%,services_diff%,scaleups_diff%,others_diff%
0,0.466696,-0.227725,-0.398825,-0.438742,-0.570074
1,0.264700,0.059308,0.023789,-0.404023,-0.479545
2,-0.250762,0.408453,0.383329,1.623109,-0.174098
3,0.265302,-0.001146,0.211252,-0.536769,-0.493371
4,0.318474,-0.127547,-0.119637,-0.348983,-0.449519
...,...,...,...,...,...
1167,-0.023179,1.681511,-1.000000,-1.000000,-0.231074
1168,-0.316225,1.298438,-1.000000,-1.000000,0.537851
1169,-0.316225,0.532292,-1.000000,-1.000000,0.922314
1170,1.051324,-1.000000,-1.000000,-1.000000,-1.000000


In [10]:
#this is a method to see if how many clusters we could potentially want. 
# Since Kmeans requires us to give the input of how many clusters we want, we can have a look at the inertia to figure out an "elbow" point in the chart. 
#it turned out not to be particularly useful but happy to hear if anyone has better ideas to figure out how many clusters we want to create


from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler #ended up not scaling the data in order to be able to talk about the findings in an easier way. 
import plotly.graph_objects as go
import numpy as np
X=df1

inertia = []
for i in range(1,14):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=14,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,14),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'})

In [32]:
#here is where i play with the number of clusers to figure out what they look like thanks to the polar chart.
import plotly.express as px

kmeans = KMeans(
        n_clusters=5, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )

kmeans.fit(X)
clusters1=pd.DataFrame(X,columns=df1.columns)
clusters1['cluster_name']=kmeans.labels_
polar=clusters1.groupby("cluster_name").mean().reset_index()
polar=pd.melt(polar,id_vars=["cluster_name"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="cluster_name", line_close=True,height=800,width=1400)
fig4.show()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Library/Frameworks/Pyth

In [33]:
#how many articles in each cluster? 
pie=clusters1.groupby('cluster_name').size().reset_index()
pie.columns=['cluster_name','value']
px.pie(pie,values='value',names='cluster_name')


In [39]:
#this is to create a frame with the specific articles per segment 
page_path = df.page_path
clusters_pages = pd.concat([page_path,clusters1],axis=1)

cluster_n =  clusters_pages['cluster_name']==1 #select whatever cluster you want

cluster_articles= clusters_pages[cluster_n]

cluster_articles



,page_path,money_diff%,early_diff%,services_diff%,scaleups_diff%,others_diff%,cluster_name
0,/articles/vc-salary-germany-europe/,0.466696,-0.227725,-0.398825,-0.438742,-0.570074,1
1,/articles/first-time-vc-funds-2022/,0.264700,0.059308,0.023789,-0.404023,-0.479545,1
3,/articles/top-seed-stage-uk-vcs/,0.265302,-0.001146,0.211252,-0.536769,-0.493371,1
4,/articles/top-series-a-investors-uk/,0.318474,-0.127547,-0.119637,-0.348983,-0.449519,1
6,/articles/pitching-downturn-founders-vcs/,0.078235,0.559425,0.191857,-0.315249,-0.443192,1
...,...,...,...,...,...,...,...
1159,/articles/fov-ventures-metaverse-vc-fund/,0.367549,0.532292,-1.000000,-1.000000,-0.423306,1
1160,/articles/green-glossary-carbon-accounting/,0.660595,0.532292,-1.000000,-1.000000,-1.000000,1
1161,/articles/harry-stebbings-140m-funds/,0.562913,0.915365,-1.000000,-1.000000,-1.000000,1
1169,/articles/sifted-summit-cheat-sheet-founders/,-0.316225,0.532292,-1.000000,-1.000000,0.922314,1


In [40]:
clusters1["cluster_name"] = clusters1["cluster_name"].astype(str) #don't want gradient colours :D 
early_money = px.scatter(clusters1, x="early_diff%", y="money_diff%", color="cluster_name",width=1200, height=800)
early_money.show()

In [42]:
early_scaleups = px.scatter(clusters1, x="early_diff%", y="scaleups_diff%", color="cluster_name",width=1200, height=800)
early_scaleups.show()

In [43]:
money_others = px.scatter(clusters1, x="money_diff%", y="others_diff%", color="cluster_name",width=1200, height=800)
money_others.show()

In [44]:
bigmatrix = px.scatter_matrix(clusters1, color="cluster_name",
width=1200, height=1600)
bigmatrix.show()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/express/_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [ ]:
#now the real anlysis starts, we will start looking at the different clusters and see if we can find similarities based on the user segments who reads them. 